In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from datetime import datetime
import requests
import os
#import schedule
import pytz
import json

In [10]:
CMT8_Pham_Van_Hai= "http://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=63ae7cfcbfd3d90017e8f422&camLocation=C%C3%A1ch%20M%E1%BA%A1ng%20Th%C3%A1ng%20T%C3%A1m%20%E2%80%93%20Ph%E1%BA%A1m%20V%C4%83n%20Hai&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8"
CMT8_Bac_Hai = "http://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=63ae798abfd3d90017e8f255&camLocation=C%C3%A1ch%20M%E1%BA%A1ng%20Th%C3%A1ng%20T%C3%A1m%20-%20B%E1%BA%AFc%20H%E1%BA%A3i&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8"
CMT8_Truong_Son = "http://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=63ae79aabfd3d90017e8f26a&camLocation=C%C3%A1ch%20M%E1%BA%A1ng%20Th%C3%A1ng%20T%C3%A1m%20-%20Tr%C6%B0%E1%BB%9Dng%20S%C6%A1n&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8"
CMT8_To_Hien_Thanh = "http://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=63ae7966bfd3d90017e8f240&camLocation=C%C3%A1ch%20M%E1%BA%A1ng%20Th%C3%A1ng%20T%C3%A1m%20-%20T%C3%B4%20Hi%E1%BA%BFn%20Th%C3%A0nh&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8"
CMT8_Hoa_Hung = "http://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=631955e7c9eae60017a1c30a&camLocation=C%C3%A1ch%20M%E1%BA%A1ng%20Th%C3%A1ng%20T%C3%A1m%20%E2%80%93%20H%C3%B2a%20H%C6%B0ng&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8"

cameras = [CMT8_Pham_Van_Hai, CMT8_Bac_Hai, CMT8_Truong_Son, CMT8_To_Hien_Thanh, CMT8_Hoa_Hung]

In [16]:
def image_processing (img):
    url = 'http://157.230.44.40:5000/process_traffic_status'
    result = requests.post(url, files={'image': img})
    result = result.json()
    return result

In [62]:
import os

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

camera_data = []
cam_locations = ['Pham Van Hai', 'Bac Hai', 'Truong Son', 'To Hien Thanh', 'Hoa Hung']
for index, (cam, ith, cam_location) in enumerate(zip(cameras, range(1,len(cameras)+1), cam_locations)):
    if ith != 1:
        driver.execute_script(f"window.open('about:blank','{str(ith)}ndtab');")
        driver.switch_to.window(f"{str(ith)}ndtab")

    driver.get(cam)
    driver.set_window_size(1000, 800)
    driver.save_screenshot("cam" + str(ith) +".png")
    with open("cam" + str(ith) +".png", "rb") as image_file:
        image_bytes = image_file.read()
    camera =  image_processing (image_bytes)
    camera['location'] = cam_location
    camera_data.append(camera)

    os.remove("cam" + str(ith) +".png")


In [67]:
Saigon_timezone = pytz.timezone('Asia/Saigon')
dt_obj = datetime.now(Saigon_timezone)
dt_obj

datetime.datetime(2024, 3, 29, 5, 16, 20, 944368, tzinfo=<DstTzInfo 'Asia/Saigon' +07+7:00:00 STD>)

In [60]:
res = camera_data[0]

traffic_json = {}
traffic_json['time'] = time
traffic_json['person'] = res['object_counts'].get('person', 0)
traffic_json['bicycle'] = res['object_counts'].get('bicycle', 0)
traffic_json['motorcycle'] = res['object_counts'].get('motorcycle', 0)
traffic_json['car'] = res['object_counts'].get('car', 0)
traffic_json['stop_sign'] = res['object_counts'].get('stop sign', 0)
traffic_json['traffic_light'] = res['object_counts'].get('traffic light', 0)
traffic_json['truck'] = res['object_counts'].get('truck', 0)
traffic_json['bus'] = res['object_counts'].get('bus', 0)
traffic_json['potted_plant'] = res['object_counts'].get('potted plant', 0)
traffic_json['traffic_status'] = res.get('traffic_status', 'Medium')
traffic_json['location'] = res.get('location', 'None')
traffic_json

{'time': <module 'time' (built-in)>,
 'person': 5,
 'bicycle': 1,
 'motorcycle': 0,
 'car': 2,
 'stop_sign': 0,
 'traffic_light': 3,
 'truck': 0,
 'bus': 0,
 'potted_plant': 0,
 'traffic_status': 'Low',
 'location': 'Pham Van Hai'}

In [61]:
camera_data[0]

{'object_counts': {'bicycle': 1,
  'car': 2,
  'fire hydrant': 1,
  'person': 5,
  'traffic light': 3},
 'timestamp': '04-04-58',
 'traffic_status': 'Low',
 'location': 'Pham Van Hai',
 'time': <module 'time' (built-in)>}

In [51]:
realtime_url = "https://api.tomorrow.io/v4/weather/realtime?location=10.7829647,106.670745&apikey=Ez2hSkCFqMrvsXGs56HWnk7eWcKGwGP8"
daytime_url = "https://api.tomorrow.io/v4/timelines?apikey=Ez2hSkCFqMrvsXGs56HWnk7eWcKGwGP8"

payload = {
    "location": "10.7829647,106.670745",
    "fields": ["temperature", "cloudBase", "cloudCeiling", "cloudCover", "dewPoint", "freezingRainIntensity", "humidity", "precipitationProbability", "pressureSurfaceLevel", "rainIntensity", "sleetIntensity", "snowIntensity", "temperatureApparent", "uvHealthConcern", "uvIndex", "visibility", "weatherCode", "windDirection", "windGust", "windSpeed"],
    "units": "metric",
    "timesteps": ["15m"],
    "startTime": "nowMinus24h",
    "endTime": "now",
    "timezone": "auto"
}
headers = {
    "accept": "application/json",
    "Accept-Encoding": "gzip",
    "content-type": "application/json"
}


headers = {"accept": "application/json"}
response = requests.get(realtime_url, headers=headers)
response_data = response.json()


response_data

{'data': {'time': '2024-03-28T21:02:00Z',
  'values': {'cloudBase': 0.26,
   'cloudCeiling': None,
   'cloudCover': 44,
   'dewPoint': 26,
   'freezingRainIntensity': 0,
   'humidity': 89,
   'precipitationProbability': 0,
   'pressureSurfaceLevel': 1010.38,
   'rainIntensity': 0,
   'sleetIntensity': 0,
   'snowIntensity': 0,
   'temperature': 28,
   'temperatureApparent': 33.57,
   'uvHealthConcern': 0,
   'uvIndex': 0,
   'visibility': 9.99,
   'weatherCode': 1101,
   'windDirection': 140,
   'windGust': 4.69,
   'windSpeed': 3.63}},
 'location': {'lat': 10.7829647, 'lon': 106.670745}}